# 02 - Loading Data

In [ ]:
import teehr.loading.nwm_point_data as nwmp
from pathlib import Path

In [ ]:
?nwmp.nwm_to_parquet

In [ ]:
# Set some notebook variables to point to the relevant study files.  
RUN = "short_range"
OUTPUT_TYPE = "channel_rt"
VARIABLE_NAME = "streamflow"

START_DATE = "2023-03-18"
INGEST_DAYS = 1

OUTPUT_ROOT = Path(Path().home(), "cache")
JSON_DIR = Path(OUTPUT_ROOT, "zarr", RUN)
OUTPUT_DIR = Path(OUTPUT_ROOT, "timeseries", RUN)

# For this simple example, we'll get data for 10 NWM reaches that coincide with USGS gauges
LOCATION_IDS = [7086109,  7040481,  7053819,  7111205,  7110249, 14299781, 14251875, 14267476,  7152082, 14828145]

In [ ]:
from dask.distributed import Client
client = Client(n_workers=8)
client

In [ ]:
%%time
nwmp.nwm_to_parquet(
    RUN,
    OUTPUT_TYPE,
    VARIABLE_NAME,
    START_DATE,
    INGEST_DAYS,
    LOCATION_IDS,
    JSON_DIR,
    OUTPUT_DIR
)

In [ ]:
!tree ~/cache/